<a href="https://colab.research.google.com/github/Mrnine12/Topicos_A/blob/main/Script_Atividade_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Introdução**

São diversos os exemplos de situações em que compras realizadas no cartão de créditos são impedidas de serem realizadas por um alerta de possível fraude no cartão. Muitas vezes, estes alertas são falsos positivos, porém para que verdadeiras fraudes sejam combatidas é de extrema importância que as empresas de cartão de crédito possam reconhecer estas transações fraudulentas, para que assim as mesmas sejam impedidas.


Os dados a serem aqui utilizados são uma amostra de um conjunto maior que contêm transações feitas por cartões de crédito em setembro de 2013 por europeus titulares de cartões de credito. A base original apresenta transações ocorridas em apenas dois dias, e se trata de um conjunto extremamente desbalanceado.

A amostra aqui utilizada, conserva essa caracteristica de desbalanceamento, e possui um total de 39992 transações.

Dessa forma, o objetivo desta análise é de identificar transações fraudulentas com o cartão de crédito, enfrentando desafios pelo alto nível de desbalanceamento da base utilizada.

**Variáveis**

Por questões de confidencialidade e segurança, não foram disponibilizados os dados reais, e sim variáveis numéricas que são resultado de uma transformação utilizando a análise de componentes principais. Porém, três variáveis originais ainda são apresentadas.

- *Tempo*: Apresenta em segundos, o tempo decorrido entre cada transação e a primeira transação no conjunto de dados.
- *Amount*: Apresenta o montante da transação.
- *Class*: É a variável resposta e assume o valor 1 em caso de fraude e 0 em 
caso contrário.

# Análise

**Pacotes**

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [29]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping

**Base utilizada**

In [30]:
creditcard = pd.read_csv("https://raw.githubusercontent.com/Mrnine12/Topicos_A/main/Atividade_I/credit_card.csv")

creditcard.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,14514,1.303975,-0.258310,-0.721635,-0.854541,1.701450,3.425969,-1.015053,0.794230,1.596253,-0.460929,0.877475,-2.511859,1.798379,1.604897,0.761679,0.810960,-0.208713,0.593591,-0.030874,0.044627,-0.119766,-0.231042,-0.050669,1.021962,0.368670,1.046266,-0.086559,-0.000983,14.95,0
1,84901,1.107639,0.069196,0.821514,1.481854,-0.591441,-0.258963,-0.173818,0.098690,0.553184,-0.133232,-0.378575,0.169830,-1.115893,0.177115,0.139224,-0.568983,0.364224,-0.991042,-0.481596,-0.243998,-0.227851,-0.514833,0.118544,0.350317,0.337361,-0.516880,0.047744,0.027315,15.00,0
2,143075,-0.941798,0.371599,1.162953,-0.804568,1.028349,0.333242,0.708923,0.196478,-0.579269,-0.753377,0.081284,0.187243,-0.224251,0.222237,-0.551577,0.431245,-0.892780,0.562988,0.467408,0.185879,0.024791,-0.166292,-0.290707,0.102435,0.690744,0.346708,-0.052592,0.048002,59.95,0
3,31415,-1.808886,0.211428,2.327059,-0.080523,-0.897376,0.297207,0.025834,-0.227003,0.101023,2.235219,-0.943096,-0.826729,0.582063,-1.168251,1.897506,-1.152253,-0.677507,2.243395,-0.575063,0.306530,-0.446711,0.189151,-0.332965,-0.171238,0.420579,-0.102430,0.437246,0.314658,100.00,0
4,77222,1.385003,-1.341675,0.427836,-1.572022,-1.428809,0.015973,-1.266940,0.025820,-1.845229,1.558713,0.489255,-0.351601,0.914950,-0.386678,0.179706,0.284073,-0.189888,1.053705,0.136131,-0.129654,-0.019739,0.187644,-0.177294,-0.494585,0.404626,-0.077278,0.036123,0.020849,79.00,0
5,38970,-1.103913,-0.030144,1.757495,-1.694520,0.274786,-1.066171,0.432315,-0.310134,-1.423179,0.523797,1.406081,0.239993,0.405224,-0.546869,-1.525588,0.684448,0.080347,-1.276018,0.526366,0.239188,0.167941,0.719872,-0.539147,0.604442,0.785351,-0.200328,0.009038,-0.131889,20.00,0
6,73171,1.367950,-0.591588,0.168180,-0.572984,-1.035432,-0.934982,-0.481263,-0.070972,-0.785585,0.798814,1.113083,-0.596007,-1.738402,0.331758,-0.770538,0.736539,0.538346,-1.037020,1.037952,-0.055680,0.196520,0.508529,-0.131773,0.542968,0.676559,-0.089498,-0.023256,-0.007846,10.00,0
7,47598,1.246482,-0.026386,0.061548,0.393117,0.375349,0.991468,-0.268131,0.204089,0.272615,-0.117991,0.116273,1.414781,1.227242,-0.279753,-0.878105,-0.121102,-0.442807,-0.218066,0.624388,-0.037806,-0.088617,0.069322,-0.248789,-1.249054,0.705629,0.531717,0.003721,-0.015040,1.00,0
8,87223,1.832598,-0.719115,-2.404618,-0.770459,0.310608,-1.185655,0.741761,-0.446728,0.445866,-0.182933,0.660160,0.904013,-0.329101,0.847972,-0.614642,-0.658958,-0.266491,-0.198398,0.928672,0.105721,0.219440,0.487048,-0.229329,-0.352939,0.308340,0.875036,-0.156122,-0.078011,160.00,0
9,157668,2.038047,0.149229,-1.493564,0.541992,0.136141,-1.183765,0.175941,-0.252149,0.505357,-0.369960,-0.159078,0.152706,-0.334666,-0.751983,0.201389,0.180484,0.678658,-0.438076,-0.139525,-0.192007,-0.334124,-0.875054,0.407086,1.043658,-0.337476,0.140677,-0.064080,-0.025846,1.98,0


Como antes comentado, a base a ser utilizada apresenta informações sobre análise de fraudes no cartão de credito, tendo como objetivo a classificação de movimentações ou compras com o cartão de credito como fraude ou não.

Acima é apresentada uma mostra dos dados, apresentando as três variaveis com "valores reais", e as demais variaveis com os valores obtidos com a utilização da Análise de Componentes Principais.

**Pré processamento**

Dando inicio a análise propriamente dita, inicialmente deve se conhecer melhor os dados. Presença de NA's, tipo de variáveis e uma análise descritiva propriamente dita é necessaria para se tomar uma melhor decisão sobre o ajuste do modelo.

In [ ]:
## Formato dos dados

creditcard.dtypes

In [ ]:
## Busca por NA's

creditcard.isnull().sum()

In [ ]:
## Analise descritiva

creditcard.iloc[:, [0, 29]].describe()

In [ ]:
## Situacao quanto ao desbalanceamento

print('Transações Não Fraudulentas: {}%'.format(round(creditcard.Class.value_counts()[0]/len(creditcard)*100.0,2)))
print('Transações Fraudulentas: {}%'.format(round(creditcard.Class.value_counts()[1]/len(creditcard)*100.0,2)))

Com essa análise inicial é possivel observar que o conjunto de dados não possui dados faltantes, e que exceto a variavel resposta que é categorica, as demais são númerica.

Observa-se a grande variabilidade para as variaveis Time e Amount, além do grande desbalanceamento para a variavel resposta, onde apenas 1.23% dos dados é de transações fraudulentas.

Para melhor utilizar os dados no futuro modelo, dada a grande variabilidade vista para as duas "variáveis reais" numericas, esta duas variaveis serão padronizadas.

Quanto ao grande desbalanceamento visto para a variavel resposta, serão utilizadas três abordagens diferentes:

- O primeiro modelo terá os dados da forma que estão com o desbalanceamento já existente;
- O segundo modelo terá a presença da categoria 1 (fraude) forçada, onde se terá uma divisão em teste e treinamento estratificada;
- Por fim, o terceiro modelo utilizará como dados para treino e teste uma amostra da base inteira, que terá todas as observações para a categoria 1 (fraude, categoria com menos observações) e uma amostra da categoria 0 (não fraude) do tamanho de observações da categoria 1.

In [ ]:
## Padronizacao de variaveis com alta variacao

padrao = RobustScaler()
creditcard['Amount'] = padrao.fit_transform(creditcard['Amount'].values.reshape(-1,1))
creditcard['Time'] = padrao.fit_transform(creditcard['Time'].values.reshape(-1,1))

creditcard.head(3)

**Modelagem**

In [18]:
def medidas(Yteste, previsoes):

  matrix_conf = confusion_matrix(Yteste, previsoes, labels = [1, 0])
  
  # Acuracia
  
  Acuracia = round(accuracy_score(Yteste, previsoes),3)
  
  # Sensibilidade
  
  Sensibilidade = round(matrix_conf[1,1]/(matrix_conf[1,1]+matrix_conf[1,0]),3)
  
  # Especificidade
  
  Especificidade = round(matrix_conf[0,0]/(matrix_conf[0,0]+matrix_conf[0,1]),3)
  
  return(Acuracia, Sensibilidade, Especificidade)

In [95]:
Time = ACC = Sen = Esp = np.repeat(0, 9)

Results = pd.DataFrame([Time, ACC, Sen, Esp],  columns = ["M1_2C_R","M1_2C_T","M1_1C_R",
                                                          "M2_2C_R","M2_2C_T","M2_1C_R",
                                                          "M3_2C_R","M3_2C_T","M3_1C_R"],
                      index = ["Time", "ACC", "Sen", "Esp"])

*Modelo 1 - Padrão*

In [96]:
Xtreino, Xteste, Ytreino, Yteste = train_test_split(creditcard.iloc[:, creditcard.columns != 'Class'],
                                                    creditcard['Class'], test_size = 0.25)

Versão 1 - Duas Camadas (relu + sigmoid)

In [97]:
modelo = Sequential()

modelo.add(Dense(5, input_dim = 30, activation = 'relu')) 
modelo.add(Dense(5, activation = 'sigmoid')) 
modelo.add(Dense(1, activation = 'sigmoid'))   

alg = Adam(learning_rate = 0.01)

modelo.compile(loss = 'binary_crossentropy', optimizer = alg, metrics = ['accuracy'])

In [ ]:
import time
start = time.time() 

parada = EarlyStopping(monitor = 'loss', patience = 5)

treinamento = modelo.fit(Xtreino, Ytreino, validation_data = (Xteste, Yteste), 
                     epochs = 25, verbose = 1, batch_size = 32, callbacks = parada)

end = time.time()
Results.iloc[0, 0] = (end - start)

In [99]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

prob = modelo.predict(Xteste)

previsoes = np.repeat(0, len(prob))

for x in range(0, len(previsoes), 1):
  if prob[x] > 0.5:
    previsoes[x] = 1
    
resul = medidas(Yteste, previsoes)

Results.iloc[1:4, 0] = resul

Versão 2 - Duas Camadas (tanh + sigmoid)

In [100]:
modelo = Sequential()

modelo.add(Dense(5, input_dim = 30, activation = 'tanh')) 
modelo.add(Dense(5, activation = 'sigmoid')) 
modelo.add(Dense(1, activation = 'sigmoid'))   

alg = Adam(learning_rate = 0.01)

modelo.compile(loss = 'binary_crossentropy', optimizer = alg, metrics = ['accuracy'])

In [ ]:
import time
start = time.time() 

parada = EarlyStopping(monitor = 'loss', patience = 5)

treinamento = modelo.fit(Xtreino, Ytreino, validation_data = (Xteste, Yteste), 
                     epochs = 25, verbose = 1, batch_size = 32, callbacks = parada)

end = time.time()
Results.iloc[0, 1] = (end - start)

In [102]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

prob = modelo.predict(Xteste)

previsoes = np.repeat(0, len(prob))

for x in range(0, len(previsoes), 1):
  if prob[x] > 0.5:
    previsoes[x] = 1
    
resul = medidas(Yteste, previsoes)

Results.iloc[1:4, 1] = resul

Versão 3 - Camada Única (relu)

In [103]:
modelo = Sequential()

modelo.add(Dense(5, input_dim = 30, activation = 'relu')) 
modelo.add(Dense(1, activation = 'sigmoid'))   

alg = Adam(learning_rate = 0.01)

modelo.compile(loss = 'binary_crossentropy', optimizer = alg, metrics = ['accuracy'])

In [ ]:
import time
start = time.time() 

parada = EarlyStopping(monitor = 'loss', patience = 5)

treinamento = modelo.fit(Xtreino, Ytreino, validation_data = (Xteste, Yteste), 
                     epochs = 25, verbose = 1, batch_size = 32, callbacks = parada)

end = time.time()
Results.iloc[0, 2] = (end - start)

In [105]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

prob = modelo.predict(Xteste)

previsoes = np.repeat(0, len(prob))

for x in range(0, len(previsoes), 1):
  if prob[x] > 0.5:
    previsoes[x] = 1
    
resul = medidas(Yteste, previsoes)

Results.iloc[1:4, 2] = resul

*Modelo 2 - Amostra Estratificada*

In [107]:
## Modelo 2

A = creditcard.query('Class == 0')
B = creditcard.query('Class == 1')

Xtreino_A, Xteste_A, Ytreino_A, Yteste_A = train_test_split(A.iloc[:, A.columns != 'Class'], A['Class'], test_size = 0.25)
Xtreino_B, Xteste_B, Ytreino_B, Yteste_B = train_test_split(B.iloc[:, B.columns != 'Class'], B['Class'], test_size = 0.25)

Xtreino = pd.concat([Xtreino_A, Xtreino_B])
Xteste = pd.concat([Xteste_A, Xteste_B])
Ytreino = pd.concat([Ytreino_A, Ytreino_B])
Yteste = pd.concat([Yteste_A, Yteste_B])

*Versão 1 - Duas Camadas (relu + sigmoid)*

In [108]:
modelo = Sequential()

modelo.add(Dense(5, input_dim = 30, activation = 'relu')) 
modelo.add(Dense(5, activation = 'sigmoid')) 
modelo.add(Dense(1, activation = 'sigmoid'))   

alg = Adam(learning_rate = 0.01)

modelo.compile(loss = 'binary_crossentropy', optimizer = alg, metrics = ['accuracy'])

In [ ]:
import time
start = time.time() 

parada = EarlyStopping(monitor = 'loss', patience = 5)

treinamento = modelo.fit(Xtreino, Ytreino, validation_data = (Xteste, Yteste), 
                     epochs = 25, verbose = 1, batch_size = 32, callbacks = parada)

end = time.time()
Results.iloc[0, 3] = (end - start)

In [110]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

prob = modelo.predict(Xteste)

previsoes = np.repeat(0, len(prob))

for x in range(0, len(previsoes), 1):
  if prob[x] > 0.5:
    previsoes[x] = 1
    
resul = medidas(Yteste, previsoes)

Results.iloc[1:4, 3] = resul

Versão 2 - Duas Camadas (tanh + sigmoid)

In [111]:
modelo = Sequential()

modelo.add(Dense(5, input_dim = 30, activation = 'tanh')) 
modelo.add(Dense(5, activation = 'sigmoid')) 
modelo.add(Dense(1, activation = 'sigmoid'))   

alg = Adam(learning_rate = 0.01)

modelo.compile(loss = 'binary_crossentropy', optimizer = alg, metrics = ['accuracy'])

In [ ]:
import time
start = time.time() 

parada = EarlyStopping(monitor = 'loss', patience = 5)

treinamento = modelo.fit(Xtreino, Ytreino, validation_data = (Xteste, Yteste), 
                     epochs = 25, verbose = 1, batch_size = 32, callbacks = parada)

end = time.time()
Results.iloc[0, 4] = (end - start)

In [115]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

prob = modelo.predict(Xteste)

previsoes = np.repeat(0, len(prob))

for x in range(0, len(previsoes), 1):
  if prob[x] > 0.5:
    previsoes[x] = 1
    
resul = medidas(Yteste, previsoes)

Results.iloc[1:4, 4] = resul

Versão 3 - Camada Única (relu)

In [117]:
modelo = Sequential()

modelo.add(Dense(5, input_dim = 30, activation = 'relu')) 
modelo.add(Dense(1, activation = 'sigmoid'))   

alg = Adam(learning_rate = 0.01)

modelo.compile(loss = 'binary_crossentropy', optimizer = alg, metrics = ['accuracy'])

In [ ]:
import time
start = time.time() 

parada = EarlyStopping(monitor = 'loss', patience = 5)

treinamento = modelo.fit(Xtreino, Ytreino, validation_data = (Xteste, Yteste), 
                     epochs = 25, verbose = 1, batch_size = 32, callbacks = parada)

end = time.time()
Results.iloc[0, 5] = (end - start)

In [119]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

prob = modelo.predict(Xteste)

previsoes = np.repeat(0, len(prob))

for x in range(0, len(previsoes), 1):
  if prob[x] > 0.5:
    previsoes[x] = 1
    
resul = medidas(Yteste, previsoes)

Results.iloc[1:4, 5] = resul

*Modelo 3 - Meio a Meio*

In [121]:
### Modelo 3

A = creditcard.query('Class == 0')
B = creditcard.query('Class == 1')

A = A.sample(n=len(B))

Xtreino_A, Xteste_A, Ytreino_A, Yteste_A = train_test_split(A.iloc[:, A.columns != 'Class'], A['Class'], test_size = 0.25)
Xtreino_B, Xteste_B, Ytreino_B, Yteste_B = train_test_split(B.iloc[:, B.columns != 'Class'], B['Class'], test_size = 0.25)

Xtreino = pd.concat([Xtreino_A, Xtreino_B])
Xteste = pd.concat([Xteste_A, Xteste_B])
Ytreino = pd.concat([Ytreino_A, Ytreino_B])
Yteste = pd.concat([Yteste_A, Yteste_B])

*Versão 1 - Duas Camadas (relu + sigmoid)*

In [122]:
modelo = Sequential()

modelo.add(Dense(5, input_dim = 30, activation = 'relu')) 
modelo.add(Dense(5, activation = 'sigmoid')) 
modelo.add(Dense(1, activation = 'sigmoid'))   

alg = Adam(learning_rate = 0.01)

modelo.compile(loss = 'binary_crossentropy', optimizer = alg, metrics = ['accuracy'])

In [ ]:
import time
start = time.time() 

parada = EarlyStopping(monitor = 'loss', patience = 5)

treinamento = modelo.fit(Xtreino, Ytreino, validation_data = (Xteste, Yteste), 
                     epochs = 25, verbose = 1, batch_size = 32, callbacks = parada)

end = time.time()
Results.iloc[0, 6] = (end - start)

In [124]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

prob = modelo.predict(Xteste)

previsoes = np.repeat(0, len(prob))

for x in range(0, len(previsoes), 1):
  if prob[x] > 0.5:
    previsoes[x] = 1
    
resul = medidas(Yteste, previsoes)

Results.iloc[1:4, 6] = resul

Versão 2 - Duas Camadas (tanh + sigmoid)

In [125]:
modelo = Sequential()

modelo.add(Dense(5, input_dim = 30, activation = 'tanh')) 
modelo.add(Dense(5, activation = 'sigmoid')) 
modelo.add(Dense(1, activation = 'sigmoid'))   

alg = Adam(learning_rate = 0.01)

modelo.compile(loss = 'binary_crossentropy', optimizer = alg, metrics = ['accuracy'])

In [ ]:
import time
start = time.time() 

parada = EarlyStopping(monitor = 'loss', patience = 5)

treinamento = modelo.fit(Xtreino, Ytreino, validation_data = (Xteste, Yteste), 
                     epochs = 25, verbose = 1, batch_size = 32, callbacks = parada)

end = time.time()
Results.iloc[0, 7] = (end - start)

In [127]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

prob = modelo.predict(Xteste)

previsoes = np.repeat(0, len(prob))

for x in range(0, len(previsoes), 1):
  if prob[x] > 0.5:
    previsoes[x] = 1
    
resul = medidas(Yteste, previsoes)

Results.iloc[1:4, 7] = resul

Versão 3 - Camada Única (relu)

In [128]:
modelo = Sequential()

modelo.add(Dense(5, input_dim = 30, activation = 'relu')) 
modelo.add(Dense(1, activation = 'sigmoid'))   

alg = Adam(learning_rate = 0.01)

modelo.compile(loss = 'binary_crossentropy', optimizer = alg, metrics = ['accuracy'])

In [ ]:
import time
start = time.time() 

parada = EarlyStopping(monitor = 'loss', patience = 5)

treinamento = modelo.fit(Xtreino, Ytreino, validation_data = (Xteste, Yteste), 
                     epochs = 25, verbose = 1, batch_size = 32, callbacks = parada)

end = time.time()
Results.iloc[0, 8] = (end - start)

In [130]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

prob = modelo.predict(Xteste)

previsoes = np.repeat(0, len(prob))

for x in range(0, len(previsoes), 1):
  if prob[x] > 0.5:
    previsoes[x] = 1
    
resul = medidas(Yteste, previsoes)

Results.iloc[1:4, 8] = resul

**Resultados**

In [131]:
Results

,M1_2C_R,M1_2C_T,M1_1C_R,M2_2C_R,M2_2C_T,M2_1C_R,M3_2C_R,M3_2C_T,M3_1C_R
Time,33.602801,33.575692,21.276658,21.441897,34.01394,17.908062,1.985963,1.953169,1.930383
ACC,0.997000,0.998000,0.998000,0.997000,0.99700,0.997000,0.931000,0.919000,0.935000
Sen,0.999000,1.000000,1.000000,1.000000,1.00000,0.999000,0.951000,0.927000,0.951000
Esp,0.831000,0.814000,0.831000,0.756000,0.79700,0.789000,0.911000,0.911000,0.919000


*Modelos*

- M1_2C_R: Modelo 1 com 2 camadas ocultas e utilizando a ligação Relu;
- M1_2C_T: Modelo 1 com 2 camadas ocultas e utilizando a ligação Tangente Hiperbolica;
- M1_1C_R: Modelo 1 com 1 camada oculta e utilizando a ligação Relu;
- M2_2C_R: Modelo 2 com 2 camadas ocultas e utilizando a ligação Relu;
- M2_2C_T: Modelo 2 com 2 camadas ocultas e utilizando a ligação Tangente Hiperbolica;
- M2_1C_R: Modelo 2 com 1 camada oculta e utilizando a ligação Relu;
- M3_2C_R: Modelo 3 com 2 camadas ocultas e utilizando a ligação Relu;
- M3_2C_T: Modelo 3 com 2 camadas ocultas e utilizando a ligação Tangente Hiperbolica;
- M3_1C_R: Modelo 3 com 1 camada oculta e utilizando a ligação Relu.


A depender de seu objetivo final, você terá um modelo para escolher, seja olhando apenas para acurácia, o tempo de modelagem ou a sensibilidade e especificidade (Vale ressaltar que nessa situação a sensibilidade está relacionada a situação de não fraude no cartão de credito, enquanto que a especificidade está relacionada a fraude no cartão de credito).

Em situações como essa, olhar apenas para acurácia é um grande erro comumente ocorrido. Dado o nivel de desbalanceamento é recomendado que se observe com mais cuidado e maior interesse para a sensibilidade e especificidade.

De forma geral, não foram encontradas grandes variações entre as arquiteturas utilizadas, exceto quanto ao tempo, onde a convergencia mais rapida fez o número de epocas necessarias ser reduzida. 

Porém, se o criterio escolhido para a escolha do modelo for o tempo, o terceiro tipo de modelo onde se utiliza de uma amostra menor dos dados é a escolha correta, dado que reduziu muito o tempo com um bom desempenho.

Além do tempo, o terceiro modelo deve ser escolhido ao se observar a especificidade, ou seja caso utilize-se o modelo com a intenção de prever novas fraudes (claramente o objetivo desta base de dados), o terceiro modelo apresenta o melhor resultado.

Caso se opte por prever com maior acertividade os casos de não fraude, um dos dois primeiros modelos podem ser utilizados com desempenho superior ao terceiro modelo.

**Conclusão Final**

Diante da modelagem obtida, o terceiro modelo aparenta ser o mais indicado e melhor ajustado para a situação em que se busca prever fraudes no cartão de credito. Além do alto nivel de especificidade, o modelo apresenta menor tempo para estimação, e utiliza menos observações para isso. Mesmo que abaixo, o modelo 3 ainda apresenta um bom desempenho para classificar as observações "não fraude", e de forma geral um bom nivel de acerto.